In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
summary = pd.read_csv("garmin_edge_820/summary.csv")   
details = pd.read_csv("garmin_edge_820/4557226804_ACTIVITY_data.csv", sep=";")

In [ ]:
details.rename(columns=lambda x: x.replace('record.', ''), inplace=True)
details['time'] = pd.to_datetime(details.pop('timestamp[s]'), unit='s').dt.time
details.set_index("time", inplace=True)

In [ ]:
hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165), (166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218), (219, 247), (248, 287), (288, float('inf'))]
# funzione che calcola le zone di frequenza cardiaca dato il battito in input

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone

In [ ]:
details['hr_zone'] = details['heart_rate[bpm]'].apply(get_zone, zones=hr_zones)
details['pwr_zone'] = details['power[watts]'].apply(get_zone, zones=power_zones)

In [ ]:
details['time_since_start'] = 1
details['time_since_start'] = details['time_since_start'].cumsum().sub(1)

In [ ]:
details.dtypes

In [ ]:
# Calcola la differenza di altitudine tra le righe adiacenti
details['altitude_diff'] = details['altitude[m]'] - details['altitude[m]'].shift(1)
details['distance_diff'] = details['distance[m]'] - details['distance[m]'].shift(1)

# Calcola la percentuale di pendenza
details['slope_percent'] = np.where(details['distance_diff'] == 0, 0, details['altitude_diff'] / details['distance_diff'] * 100)

In [ ]:
details.isna().sum()

### Data Visualization

In [ ]:
details.plot.scatter(y="cadence[rpm]", x="heart_rate[bpm]")

In [ ]:
details.head(100).plot.line(x='time_since_start', y='slope_percent')